In [1]:
from Quantamental_functions import *

%load_ext autoreload
%autoreload 2

In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_columns = 99999

In [3]:
cik = "0000042888"
form_type = "10-Q"
dateb = "20240101"
ticker = "GGG"
bond_ticker = "^IRX"
start_date = datetime(2000, 1, 1)
end_date = datetime(2024, 4, 1)
columns_to_check = {"marketable securities": 0, "accrued expenses and other": 0, "depreciation and amortization": 0, "tangible assets": 0}
accession_numbers = get_accession_numbers(cik, form_type, dateb)
urls = generate_edgar_urls(cik, accession_numbers, ticker, dateb)
bond_yield = get_data(bond_ticker, start_date, end_date) / 100
share_price = get_data(ticker, start_date, end_date)
company_info = fetch_company_info()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
consolidated_cf, consolidated_bs, consolidated_is = process_urls(urls)
cf_df = transpose_df(consolidated_cf)
bs_df = transpose_df(consolidated_bs)
is_df = transpose_df(consolidated_is)
cf_df, bs_df, is_df = filter_columns(cf_df, bs_df, is_df)
is_df['nshares'] = (is_df['net income'] / is_df['diluted earnings per share']).astype(int)
combo_df = pd.concat([cf_df, bs_df, is_df], axis=1)
combo_df = combo_df.sort_index(axis=1)
combo_df_dates = combo_df.index.to_series().unique()
combo_df = combo_df.assign(bond_yield=combo_df.index.to_series().map(lambda date: bond_yield.asof(date)))
combo_df = combo_df.assign(share_price=combo_df.index.to_series().map(lambda date: share_price.asof(date)))

# Cash Flow Statement

In [5]:
cf_df.head()

,capex,cash flow from financing,cash flow from investing,cash flow from operations,change in cash,depreciation and amortization
Date,,,,,,
01-04-2022,-47093.00,-202852.00,-72243.00,31325.00,-243815.00,15570.00
01-07-2022,-88861.00,-230341.00,-114554.00,134985.00,-210943.00,31453.00
30-09-2022,-147193.00,-305846.00,-172966.00,271552.00,-209475.00,48223.00
31-03-2023,-38290.00,3319.00,-38642.00,90829.00,56117.00,17987.00
30-06-2023,-92232.00,-7988.00,-93172.00,282323.00,181437.00,36117.00


# Balance Sheet

In [6]:
bs_df.head()

,accounts payable,accounts receivable,cash and cash equivalents,goodwill,inventories,long term debt,operating leases,retained earnings,tangible assets,total assets,total current assets,total current liabilities,total liabilities and equity,total shareholders equity
Date,,,,,,,,,,,,,,
01-04-2022,88108.00,339407.00,380487.00,371401.00,436741.00,75000.00,31228.00,841503.00,487805.00,2319445.00,1197220.00,402313.00,2319445.00,1690256.00
01-07-2022,85452.00,373141.00,413359.00,365491.00,450974.00,75000.00,31055.00,912864.00,516156.00,2404278.00,1276476.00,424058.00,2404278.00,1757274.00
30-09-2022,99025.00,364073.00,414827.00,359645.00,462220.00,75000.00,30996.00,950414.00,562807.00,2438200.00,1285903.00,430768.00,2438200.00,1788311.00
31-03-2023,80375.00,356473.00,395313.00,370103.00,497242.00,75000.00,29742.00,1059980.00,633482.00,2546263.00,1288461.00,381698.00,2546263.00,1986343.00
30-06-2023,81540.00,365818.00,520633.00,371880.00,479095.00,0.00,29158.00,1154453.00,680040.00,2707725.00,1410453.00,489338.00,2707725.00,2116307.00


# Income Statement

In [7]:
is_df.head()

,basic earnings per share,cost of sales,diluted earnings per share,income before income taxes,net income,operating income,total revenues,nshares
Date,,,,,,,,
01-04-2022,0.59,239810.00,0.58,122923.00,100843.00,128363.00,494285.00,173867
01-07-2022,0.69,279487.00,0.68,146347.00,117378.00,148680.00,548547.00,172614
30-09-2022,0.69,284556.00,0.67,142473.00,116232.00,143149.00,545644.00,173480
31-03-2023,0.77,244506.00,0.75,157350.00,129166.00,156668.00,529646.00,172221
30-06-2023,0.80,268229.00,0.78,159619.00,134268.00,157052.00,559644.00,172138


# Analysis

In [8]:
combo_df["eps growth rate"] = combo_df['diluted earnings per share'].pct_change().fillna(0)
combo_df["intrinsic value"] = combo_df['diluted earnings per share'] * (8.5 + 2 * combo_df["eps growth rate"]) * 4.4 / combo_df['bond_yield']
combo_df["owner earnings"] = combo_df["net income"] - combo_df["capex"]
for column in columns_to_check:
    if column not in combo_df.columns:
        combo_df[column] = columns_to_check[column]
combo_df["invested capital"] = (combo_df["total assets"] - combo_df["cash and cash equivalents"] - combo_df["marketable securities"] + combo_df["accounts payable"] + combo_df["accrued expenses and other"])
combo_df["roic"] = combo_df["owner earnings"] / combo_df["invested capital"]
combo_df["current ratio"] = combo_df["total current assets"] / combo_df["total current liabilities"]
combo_df["net current assets"] = combo_df["total current assets"] - combo_df["total current liabilities"]
combo_df["bvps"] = combo_df["total shareholders equity"] / combo_df["nshares"]
combo_df["pe ratio"] = combo_df["share_price"] / combo_df["diluted earnings per share"]
combo_df["pb ratio"] = combo_df["share_price"] / (combo_df["total shareholders equity"] / combo_df["nshares"])
combo_df["multiplier pb"] = combo_df["pe ratio"] * combo_df["pb ratio"]
combo_df["margin safety"] = 1 - (combo_df["share_price"] / combo_df["intrinsic value"])
combo_df["total debt"] = combo_df["total liabilities and equity"] - combo_df["total shareholders equity"]
combo_df["tbvps"] = combo_df["tangible assets"] / combo_df["nshares"]
combo_df["ncav"] = (combo_df["total current assets"] - combo_df["total debt"]) / combo_df["nshares"]

In [12]:
combo_df.head()

,accounts payable,accounts receivable,basic earnings per share,capex,cash and cash equivalents,cash flow from financing,cash flow from investing,cash flow from operations,change in cash,cost of sales,depreciation and amortization,diluted earnings per share,goodwill,income before income taxes,inventories,long term debt,net income,nshares,operating income,operating leases,retained earnings,tangible assets,total assets,total current assets,total current liabilities,total liabilities and equity,total revenues,total shareholders equity,bond_yield,share_price,eps growth rate,intrinsic value,owner earnings,marketable securities,accrued expenses and other,invested capital,roic,current ratio,net current assets,bvps,pe ratio,pb ratio,multiplier pb,margin safety,total debt,tbvps,ncav
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01-04-2022,88108.00,339407.00,0.59,-47093.00,380487.00,-202852.00,-72243.00,31325.00,-243815.00,239810.00,15570.00,0.58,371401.00,122923.00,436741.00,75000.00,100843.00,173867,128363.00,31228.00,841503.00,487805.00,2319445.00,1197220.00,402313.00,2319445.00,494285.00,1690256.00,0.00,78.89,0.00,27115.00,147936.00,0,0,2027066.00,0.07,2.98,794907.00,9.72,136.02,8.11,1103.78,1.00,629189.00,2.81,3.27
01-07-2022,85452.00,373141.00,0.69,-88861.00,413359.00,-230341.00,-114554.00,134985.00,-210943.00,279487.00,31453.00,0.68,365491.00,146347.00,450974.00,75000.00,117378.00,172614,148680.00,31055.00,912864.00,516156.00,2404278.00,1276476.00,424058.00,2404278.00,548547.00,1757274.00,0.00,77.05,0.17,30072.41,206239.00,0,0,2076371.00,0.10,3.01,852418.00,10.18,113.31,7.57,857.58,1.00,647004.00,2.99,3.65
30-09-2022,99025.00,364073.00,0.69,-147193.00,414827.00,-305846.00,-172966.00,271552.00,-209475.00,284556.00,48223.00,0.67,359645.00,142473.00,462220.00,75000.00,116232.00,173480,143149.00,30996.00,950414.00,562807.00,2438200.00,1285903.00,430768.00,2438200.00,545644.00,1788311.00,0.03,59.95,-0.01,785.26,263425.00,0,0,2122398.00,0.12,2.99,855135.00,10.31,89.48,5.82,520.37,0.92,649889.00,3.24,3.67
31-03-2023,80375.00,356473.00,0.77,-38290.00,395313.00,3319.00,-38642.00,90829.00,56117.00,244506.00,17987.00,0.75,370103.00,157350.00,497242.00,75000.00,129166.00,172221,156668.00,29742.00,1059980.00,633482.00,2546263.00,1288461.00,381698.00,2546263.00,529646.00,1986343.00,0.05,73.01,0.12,628.97,167456.00,0,0,2231325.00,0.08,3.38,906763.00,11.53,97.35,6.33,616.22,0.88,559920.00,3.68,4.23
30-06-2023,81540.00,365818.00,0.80,-92232.00,520633.00,-7988.00,-93172.00,282323.00,181437.00,268229.00,36117.00,0.78,371880.00,159619.00,479095.00,0.00,134268.00,172138,157052.00,29158.00,1154453.00,680040.00,2707725.00,1410453.00,489338.00,2707725.00,559644.00,2116307.00,0.05,86.35,0.04,571.44,226500.00,0,0,2268632.00,0.10,2.88,921115.00,12.29,110.71,7.02,777.55,0.85,591418.00,3.95,4.76


In [9]:
user_title = input("Enter the company name: ").upper()
filtered_companies = company_info[company_info['title'].str.contains(user_title, na=False)]
filtered_companies

,cik_str,ticker,title
733,42888,GGG,GRACO INC


In [14]:
company_classification = classify_stock_relaxed(combo_df, combo_df['share_price'], ticker, multiple=False)
print(f'{ticker} is classified as {company_classification[0]}')

GGG is classified as Defensive
